In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv('.\datasets\current_data_without_preproc.csv')
df  = df.drop(columns=["Unnamed: 0.1","Unnamed: 0"])
# age_df = df["Age"].apply(np.floor)
# df["Age"] = df[["Age"]].replace(age_df[["Age"]], inplace=True)
# df
#print(df["Age"])
df.info()

numeric_data = df.select_dtypes(include=[np.number])
categorical_data = df.select_dtypes(exclude=[np.number])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4313 entries, 0 to 4312
Data columns (total 50 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   EmployeeID                                                4313 non-null   object 
 1   Division                                                  4313 non-null   object 
 2   Department                                                4313 non-null   object 
 3   CostCenterName                                            4313 non-null   object 
 4   CostCenterCode                                            4313 non-null   object 
 5   Location                                                  4313 non-null   object 
 6   ResourceLevel                                             4313 non-null   int64  
 7   JobTitle                                                  4313 non-null   object 
 8   JobFunction       

In [4]:
def df_info(input_df):
    df_data = {}
    for column in list(input_df):
        data_to_append = {}
        # first sort through what type it is 
        ## Numerical data type workflow
        ## what if it's int32 or other variations? should i jsut convert this into a string then run .str.isnumeric operation on it?
        # using isinstance(x,int) would not work in pandas because pandas doesnt treat an int in the same way as in native python

        # Method 1 => hard code and account for all the possible dtypes 
        # Method 2 => convert dtypes to only int64 and float64, and manipulate from there

        if ('int' in input_df[f"{column}"].dtype) or ('float' in input_df[f"{column}"].dtype):
            data_to_append["Data Type"] = "Numerical"
            
            # range
            max_value = input_df[f"{column}"].max()
            min_value = input_df[f"{column}"].min()
            data_to_append["Range"] = str(min_value) + "-" + str(max_value)

        elif input_df[f"{column}"].dtype == "str" or input_df[f"{column}"].dtype == "O":
            data_to_append["Data Type"] = "Categorical"
            number_of_unique_values = input_df[f"{column}"].nunique()
            data_to_append["Unique Values"] = number_of_unique_values

        #datetime type
        elif input_df[f"{column}"].dtype == "datetime64":
            data_to_append["Data Type"] = "Date Time"

        elif input_df[f"{column}"].dtype == "bool":
            data_to_append["Data Type"] = "Boolean"

    # from there => add more details for each column respectively
        df_data[f"{column}"] = data_to_append
    
    return df_data

#print(df_info(df))

# output1 = {
#     "col a": {
#         "data type": "numeric/categorical/datetime" 
#         "if numeric - give range"
#         "if categorical - give number of unique values (i.e categories) " 
#     }
#     }

In [5]:
temp_df = df.copy(deep=True)
temp_df = temp_df[["Division", "Voluntary"]].dropna()
temp_df

,Division,Voluntary
0,Art Division,Yes
5,Game Division,No
13,GA Division,Yes
15,Game Division,Yes
33,GA Division,Yes
...,...,...
4306,Art Division,Yes
4307,Art Division,Yes
4308,Art Division,Yes
4310,Game Division,Yes


In [24]:
# FEATURES = ['Division','Department','CostCenterName','Location','ResourceLevel','JobTitle','JobFunction','Gender','Grade','MgrResourceLevel','Avg annual bonus vs monthly base salary (previous year) ','Avg quarterly people review rating (last 3 times)','Avg Extra time hours/week(6 mths)','Avg Idle day ratio/month (3 mths)','Avg Inefficiency/month (3 mths)','Avg Occupation ratio/month (3 mths)','Job Satisfaction','Personal Development (satisfaction)','Support from Company','Last quarterly people review rating','Last annual performance rating','Service Years','Promotion frequency/last 2 yrs.','Avg Salary CR/month(6 mths)','Salary adjustment frequency present year','Salary increment Delta vs last year (%)','Age'
# ]
#USING these features to predict the status

def one_dimensional_quantitative_analysis_on_status(comparison_col_name, sort_by_col_name="Number_of_People_who_left"):
    # df.grade
    test_df = df.loc[:, [f"{comparison_col_name}", "Status"]]
    test_df = test_df.assign(
        Number_of_People_who_left = np.where(test_df[["Status"]] == 2, 1, 0),
        Number_of_People_who_stayed = np.where(test_df[["Status"]] == 3, 1, 0),
        ).groupby([f"{comparison_col_name}"]).agg({f"{sort_by_col_name}":sum, "Number_of_People_who_stayed":sum}
    )
    test_df.sort_values(by=[f"{sort_by_col_name}"], ascending=False, inplace=True)
    #print(test_df.dtypes)
    #sns.catplot(kind="bar", x=f"{comparison_col_name}", y=f"{sort_by_col_name}", data=test_df)
    print(test_df) 

def two_dimensional_quantitative_analysis_on_status(primary_comparison_col, secondary_comparison_col):
    test_df = df.loc[:, [f"{primary_comparison_col}", f"{secondary_comparison_col}" ,  "Status"]]
    test_df = test_df.assign(
        Number_of_People_who_left = np.where(test_df[["Status"]] == 2, 1, 0),
        
        ).groupby([f"{primary_comparison_col}", f"{secondary_comparison_col}"]).agg({"Number_of_People_who_left":sum}
    )
    test_df.sort_values(by=[f"{primary_comparison_col}"], ascending=True, inplace=True)
    print(test_df)


#one_dimensional_quantitative_analysis_on_status("Division")

#two_dimensional_quantitative_analysis_on_status("Division", "Gender")
combined_col = []

def viz(input_df_viz: str, pri_cols_viz: list, sec_col_viz: list, val_col_viz: list):
    print("Hi")
    pass

def table_calc(input_df_calc: str, pri_cols_calc: list,sec_col_calc: list, val_col_calc: list):
    try:
        group_by_col = pri_cols_calc + sec_col_calc
        combined_col = group_by_col + val_col_calc
    except Exception as e:
        print("Error was: " + str(e))

    test_df = input_df_calc.loc[:, combined_col]

    # for loop for val cols
    for val in val_col_calc:
        # for loop to generate cols for val cols
        unique_vals = []
        new_col_name = ""
        unique_vals = list(set(df[val])) # gives me the names of the unique vals within the values col
        num_unique_vals = len(unique_vals)

        for i in range(num_unique_vals):
         
            eval_col = np.where(test_df[val] == unique_vals[i], 1, 0)# unique vals is status, voluntary words
            # creating the name of the col
            new_col_name = f"Count_of_{val}:{unique_vals[i]}"
            # creating the col in the df
            test_df[new_col_name] = eval_col.flatten()
            test_df[new_col_name] = test_df.groupby(group_by_col)[new_col_name].transform("sum")
        test_df = test_df.drop([val], axis=1)
            
    test_df = test_df.drop_duplicates(group_by_col)
    # print(test_df.head())
    viz(test_df, pri_cols_calc, sec_col_calc, val_col_calc)

pri_cols = ["Division"]
sec_col = ["Location"]
list_of_val_cols = ["Status"]

def initialisation(calc_needed: bool, input_df_initialisation: str, pri_cols_initialisation: list, sec_col_initialisation: list, val_col_initialisation: list):
    if calc_needed == True:
        f = table_calc(input_df_initialisation, pri_cols_initialisation, sec_col_initialisation, val_col_initialisation)
        #standardize output to make things easier for viz function

    else:
        pass
        # run regular viz function

# initialisation -> table calc -> viz
# initialisation -> viz

numeric_data = df.select_dtypes(include=[np.number])
categorical_data = df.select_dtypes(exclude=[np.number])

initialisation(1, df, pri_cols, sec_col, list_of_val_cols) # only this will not have an int

Hi


In [ ]:
def viz():
    if need_to_make_table:
        df=make_table()
        plot_x(df, ...)
    else:
        plot_x

In [7]:
# FEATURES = ['Division','Department','CostCenterName','Location','ResourceLevel','JobTitle','JobFunction','Gender','Grade','MgrResourceLevel','Avg annual bonus vs monthly base salary (previous year) ','Avg quarterly people review rating (last 3 times)','Avg Extra time hours/week(6 mths)','Avg Idle day ratio/month (3 mths)','Avg Inefficiency/month (3 mths)','Avg Occupation ratio/month (3 mths)','Job Satisfaction','Personal Development (satisfaction)','Support from Company','Last quarterly people review rating','Last annual performance rating','Service Years','Promotion frequency/last 2 yrs.','Avg Salary CR/month(6 mths)','Salary adjustment frequency present year','Salary increment Delta vs last year (%)','Age'
# ]

# hue="species" - draw multiple histograms within the same graph
#date_time_col = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Data Transformation
test_df1 = df.copy(deep=True)
test_df1["Status_Count"] = test_df1.groupby(["Location"])["Status"].transform("sum")
test_df1 = test_df1.drop_duplicates(["Location"])
test_df1["Index"] = range(1, len(test_df1) + 1)
print(test_df1[["Index", "Location", "Status_Count"]].head(5))

### Distribution Plots
## High level plots:
# hist_plot = sns.displot(data=test_df1, x="Division", y="Status_Count", kind="hist", hue="Location", palette="bright")
# kde_plot = sns.displot(data=test_df1, x="Index", y="Status_Count", kind="kde") # kde x axis must be numerical / datetime 
# for hist => hue will be a sub group by col that can be inputted

## Low level plots: 
# ecdf_plot = sns.displot(data=test_df1, x="Status_Count", kind="ecdf", hue="Status", palette="bright") # only one variable needed => a probability distribution is plotted. ecdf vs cdf: cdf is based on hypothetitical but ecdf is empirical(actual) data

### Categorical plots
## High level plots:
# bar_plot = sns.catplot(data=test_df1, x="Division", y="Status_Count", kind="bar") #Error bars give the precision of the measurement 
# box_plot = sns.catplot(data=test_df1, x="Division", y="Status_Count", kind="box", hue="Status", palette="bright")

## Low level plots:
# swarm_plot = sns.catplot(data=test_df1, x="Status_Count", y="Location", kind="swarm", hue="Division", palette="bright")

### Relational plots
## High level plots:
# scatter_plot = sns.relplot(data=test_df1, x="Service Years", y="Status_Count", kind="scatter", hue="Division", palette="bright")
# line_plot = sns.relplot(data=test_df1, x="Status_Count", y="Location", kind="line", hue="Division", palette="bright")

### Regression plots
## High level plots
# lm_plot = sns.lmplot(data=test_df1, x="Index", y="Status_Count", hue="Division", palette="bright")

     Index Location  Status_Count
0        1      SHA          2328
31       2      SGP           181
130      3      CDU          3058
169      4      PAR           364
375      5      XIY          1145


In [ ]:
# Poll list:
# 1) Games Night (offline or online)
# 2) Badminton 
# 3) Rock climbing 
# 4) 
